In [1]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, text, MetaData, Table

db_url = URL.create(
    drivername="mysql+pymysql",
    username="teamx",
    password="#C!D123^-c12",
    host="112.125.88.107",
    port=5906,
    database="TeamX_BIGAI",
    query={"charset": "utf8mb4"},
)

# db_url = URL.create(
#     drivername="mysql+pymysql",
#     username="agentictrl",
#     password="`1qaz~!QAZ",
#     host="112.125.88.107",
#     port=5906,
#     database="BIGAI",
#     query={"charset": "utf8mb4"},
# )

engine = create_engine(db_url, pool_pre_ping=True)

In [2]:

metadata = MetaData()

# 反射全部表结构
metadata.reflect(bind=engine)

print("数据库 BIGAI 包含以下表：")
for tbl in metadata.sorted_tables:
    print(f"\n🗂  表：{tbl.name}")
    for col in tbl.columns:
        print(f"   - {col.name} ({col.type}) {'PK' if col.primary_key else ''}")

数据库 BIGAI 包含以下表：

🗂  表：checkpoint
   - id (BIGINT) PK
   - name (VARCHAR(50)) 
   - version (VARCHAR(50)) 
   - run_id (VARCHAR(191)) 
   - status (VARCHAR(20)) 
   - path (VARCHAR(255)) 
   - source (VARCHAR(50)) 
   - operator (VARCHAR(50)) 
   - remark (VARCHAR(1024)) 
   - config_yaml (TEXT) 
   - created_at (TIMESTAMP) 
   - updated_at (TIMESTAMP) 
   - deleted_at (TIMESTAMP) 
   - started_at (TIMESTAMP) 
   - finished_at (TIMESTAMP) 

🗂  表：current_model
   - id (BIGINT) PK
   - checkpoint_id (BIGINT) 
   - version (VARCHAR(50)) 
   - path (VARCHAR(255)) 
   - status (VARCHAR(20)) 
   - created_at (TIMESTAMP) 
   - updated_at (TIMESTAMP) 
   - activated_by (VARCHAR(50)) 
   - remark (VARCHAR(1024)) 

🗂  表：dataset_usage_events
   - id (BIGINT) PK
   - trajectory_id (VARCHAR(128) COLLATE "utf8mb4_unicode_ci") 
   - run_id (VARCHAR(128) COLLATE "utf8mb4_unicode_ci") 
   - model_version (VARCHAR(512) COLLATE "utf8mb4_unicode_ci") 
   - used_delta (INTEGER) 
   - event_type (ENUM) 
   

In [3]:
# run_id="results/pass16_20250901_train15_pass16_gpu2_env20_vllm_logp_maxstep15"# results/test_for_train_instruction
run_id="results/pass8_20250909_libreoffice_calc_pass8_gpu2_env18_vllm_logp_maxstep30"
with engine.connect() as conn:
    result = conn.execute(text(f"SELECT count(*) ct, avg(num_chunks), model_version FROM `rollout_run` where run_id='{run_id}' group by model_version"))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"rollout_run 共 {len(rows)} 行")
# 可选：转成 DataFrame 方便查看
import pandas as pd; pd.DataFrame(rows)

rollout_run 共 1 行


,avg(num_chunks),ct,model_version
0,14.8542,48,/capacity/userdata/vcfenxd75jiv/shichenrui/ui_...


In [ ]:
# results/pass16_20250901_train15_pass16_gpu2_env20_vllm_logp_maxstep15

# results/pass16_20250901_train15_pass16_gpu2_env20_vllm_logp_maxstep15

In [ ]:
import pandas as pd

# 查看run_id已采样的数据
# run_id="results/test_for_train_pass8_gpu7_env69_20250821_2333" # results/test_for_train_instruction  results/pass@32_trainset90
run_id="results/pass8_20250907_train_acc_lt_0.7_pass8_gpu2_env18_vllm_logp_maxstep15" # results/test_for_train_instruction  results/pass@32_trainset90
with engine.connect() as conn:
    result = conn.execute(text(f"SELECT * FROM `rollout_run` where run_id='{run_id}' order by create_at desc"))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"rollout_run 共 {len(rows)} 行")
# 可选：转成 DataFrame 方便查看
df = pd.DataFrame(rows)
if "task_id" in df.columns and "reward" in df.columns:
    avg_rewards = df.groupby("task_id")["reward"].mean()
    traj_counts = df.groupby("task_id")["trajectory_id"].nunique()
    summary_df = pd.DataFrame({
        "avg_reward": avg_rewards,
        "traj_count": traj_counts
    })
    print("每个 task 的统计：")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(summary_df)
else:
    print("数据中缺少 task_id 或 reward 字段")


import pandas as pd; pd.DataFrame(rows)


rollout_run 共 155 行
每个 task 的统计：


,avg_reward,traj_count
task_id,,
06ca5602-62ca-47f6-ad4f-da151cde54cc,1.000000,2
06fe7178-4491-4589-810f-2e2bc9502122,1.000000,3
0810415c-bde4-4443-9047-d5f70165a697,1.000000,1
0e47de2a-32e0-456c-a366-8c607ef7a9d2,0.000000,1
0f84bef9-9790-432e-92b7-eece357603fb,0.000000,2
12086550-11c0-466b-b367-1d9e75b3910e,1.000000,2
12382c62-0cd1-4bf2-bdc8-1d20bf9b2371,0.000000,5
1334ca3e-f9e3-4db8-9ca7-b4c653be7d17,1.000000,3
13584542-872b-42d8-b299-866967b5c3ef,0.000000,1


,create_at,id,instruction,model_version,num_chunks,reward,run_id,split_dir,task_id,trace_id,trajectory_id,used
0,2025-09-04 18:03:45,378865,Cross-check the invoices with the bank stateme...,/root/verl/checkpoints/verl_osworld_grpo/oswor...,30,0.0,results/pass8_20250904_train90_pass8_gpu3_env3...,None,337d318b-aa07-4f4f-b763-89d9a2dd013f,trace-8fdb023d124b-1756979455,337d318b-aa07-4f4f-b763-89d9a2dd013f_trace-8fd...,0
1,2025-09-04 18:03:42,378864,Cross-check the invoices with the bank stateme...,/root/verl/checkpoints/verl_osworld_grpo/oswor...,30,0.0,results/pass8_20250904_train90_pass8_gpu3_env3...,None,337d318b-aa07-4f4f-b763-89d9a2dd013f,trace-a64304a52562-1756979455,337d318b-aa07-4f4f-b763-89d9a2dd013f_trace-a64...,0
2,2025-09-04 18:03:30,378862,Create two local folders in Thunderbird for me...,/root/verl/checkpoints/verl_osworld_grpo/oswor...,30,0.0,results/pass8_20250904_train90_pass8_gpu3_env3...,None,a10b69e1-6034-4a2b-93e1-571d45194f75,trace-3dbee589c39c-1756979455,a10b69e1-6034-4a2b-93e1-571d45194f75_trace-3db...,0
3,2025-09-04 18:03:13,378861,Can you assist me in shifting the text box to ...,/root/verl/checkpoints/verl_osworld_grpo/oswor...,30,0.0,results/pass8_20250904_train90_pass8_gpu3_env3...,None,e2dd0213-26db-4349-abe5-d5667bfd725c,trace-e4b7128c35d5-1756979455,e2dd0213-26db-4349-abe5-d5667bfd725c_trace-e4b...,0
4,2025-09-04 18:03:12,378860,Could you assist me in placing my photo on the...,/root/verl/checkpoints/verl_osworld_grpo/oswor...,9,1.0,results/pass8_20250904_train90_pass8_gpu3_env3...,None,77b8ab4d-994f-43ac-8930-8ca087d7c4b4,trace-215e9d8ac61c-1756979890,77b8ab4d-994f-43ac-8930-8ca087d7c4b4_trace-215...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
150,2025-09-04 17:34:08,378495,"Change the font size of the content to 12, and...",/root/verl/checkpoints/verl_osworld_grpo/oswor...,13,1.0,results/pass8_20250904_train90_pass8_gpu3_env3...,None,a434992a-89df-4577-925c-0c58b747f0f4,trace-5da0c7bbdddb-1756978296,a434992a-89df-4577-925c-0c58b747f0f4_trace-5da...,0
151,2025-09-04 17:33:59,378494,"Create a new sheet named ""Sheet2"" and merge ce...",/root/verl/checkpoints/verl_osworld_grpo/oswor...,10,1.0,results/pass8_20250904_train90_pass8_gpu3_env3...,None,1d17d234-e39d-4ed7-b46f-4417922a4e7c,trace-66f17f58df15-1756978296,1d17d234-e39d-4ed7-b46f-4417922a4e7c_trace-66f...,0
152,2025-09-04 17:33:53,378492,Can you make the video fill up the whole scree...,/root/verl/checkpoints/verl_osworld_grpo/oswor...,3,0.0,results/pass8_20250904_train90_pass8_gpu3_env3...,None,8d9fd4e2-6fdb-46b0-b9b9-02f06495c62f,trace-fb9bb52f4ab2-1756978296,8d9fd4e2-6fdb-46b0-b9b9-02f06495c62f_trace-fb9...,0
153,2025-09-04 17:33:53,378493,Can you make the video fill up the whole scree...,/root/verl/checkpoints/verl_osworld_grpo/oswor...,3,0.0,results/pass8_20250904_train90_pass8_gpu3_env3...,None,8d9fd4e2-6fdb-46b0-b9b9-02f06495c62f,trace-9c93101af74b-1756978296,8d9fd4e2-6fdb-46b0-b9b9-02f06495c62f_trace-9c9...,0


In [8]:
# 查看run_id已保存的checkpoints
run_id="results/pass16_20250902_train90_pass16_gpu2_env20_vllm_logp_maxstep50" # results/test_for_train_instruction  results/pass@32_trainset90
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM `checkpoint` where run_id = '{run_id}' "))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"checkpoint 共 {len(rows)} 行")
# 可选：转成 DataFrame 方便查看
print(rows[0])
import pandas as pd; pd.DataFrame(rows).head()

checkpoint 共 0 行


IndexError: list index out of range


# 从rollout_run中删除某run_id的数据
run_id="results/test_for_train_pass8_gpu7_env69_20250821_2333"
with engine.begin() as conn:
    will = conn.execute(
        text("SELECT COUNT(*) FROM `rollout_run` WHERE `run_id` = :rid"),
        {"rid": run_id}
    ).scalar_one()
    print(f"将删除 {will} 行（run_id = {run_id}）")

    res = conn.execute(
        text("DELETE FROM `rollout_run` WHERE `run_id` = :rid"),
        {"rid": run_id}
    )
    print(f"✅ 已删除 {res.rowcount} 行")

In [19]:
# run_id="results/test_for_train_pass8_gpu8_env77_20250817_1345"
with engine.connect() as conn:
    cnt = conn.execute(
        text("""
            SELECT COUNT(*) FROM `rollout_run`
            WHERE `run_id` = :{run_id} AND `used` != 0
        """),
        {"rid": run_id},
    ).scalar_one()
print(f"确认 used!=0 的行数：{cnt}")

with engine.begin() as conn:
    res = conn.execute(
        text("""
            UPDATE `rollout_run`
            SET `used` = 0
            WHERE `run_id` = :{run_id}
        """),
        {"rid": run_id},
    )
    print(f"✅ 更新行数: {res.rowcount}")

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ':{run_id} AND `used` != 0' at line 2")
[SQL: 
            SELECT COUNT(*) FROM `rollout_run`
            WHERE `run_id` = :{run_id} AND `used` != 0
        ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
# 更新used = 0 数据的model version
run_id="results/pass@32_trainset90"
model_version = "/workspace/computer-use/verl/checkpoints/verl_osworld_grpo/osworld_all_feasible_reward_script_grpo_k8s_0813_h8zdohoq/global_step_10/actor/huggingface"
model_version = "/capacity/userdata/vcfenxd75jiv/shichenrui/ui_tars/ByteDance-Seed/UI-TARS-1.5"
with engine.connect() as conn:
    cnt, sample = conn.execute(
        text("""
            SELECT COUNT(*) AS cnt, MAX(`model_version`) AS sample_version
            FROM `rollout_run`
            WHERE `run_id` = :rid AND `used` = 0
        """),
        {"rid": run_id},
    ).one()
print(f"used=0 的剩余行数: {cnt}，示例版本: {sample}")

with engine.begin() as conn:
    res = conn.execute(
        text("""
            UPDATE `rollout_run`
            SET `model_version` = :mv
            WHERE `run_id` = :rid AND `used` = 0
        """),
        {"mv": model_version, "rid": run_id},
    )
    print(f"✅ 更新行数: {res.rowcount}")

In [ ]:
# 统计run_id validation 结果

import numpy as np
import json
import pandas as pd
# val_trainset90_px_08220031_step10
# run_ids = ["results/val_train150_uitars_maxstep50", 
#            "results/val_trainset90_px_08220031_step10", 
#            "results/val_trainset90_px_08220031_step20", 
#            "results/val_trainset90_px_08220031_step30", 
#            "results/val_trainset90_px_08220031_step35",
#            "results/val_trainset90_px_08220031_step40"]
run_ids = ["results/pass16_20250825_train15_pass16_gpu4_env36_kl"]

with open("../evaluation_examples/test_trainset_15.json", "r", encoding="utf-8") as f:
    mapping = json.load(f)
eval_task_ids = set()
for _, lst in mapping.items():
    eval_task_ids.update(lst)
    
placeholders = ", ".join([f":id{i}" for i in range(len(run_ids))]) or ":id0"
sql = text(f"""
    SELECT reward, run_id, task_id
    FROM rollout_run
    WHERE run_id IN ({placeholders})
""")
params = {f"id{i}": rid for i, rid in enumerate(run_ids)} or {"id0": None}

with engine.begin() as conn:
    df = pd.read_sql(sql, conn, params=params)

def agg_by_run_id(df: pd.DataFrame) -> pd.DataFrame:
    x = df["reward"].fillna(0)
    reward_non_neg1 = np.where(x.eq(-1), 0, x).astype(float)

    df_calc = pd.DataFrame({
        "run_id": df["run_id"].values,
        "reward_non_neg1": reward_non_neg1
    })

    total_rows = df_calc.groupby("run_id").size().rename("total_rows")
    total_score = df_calc.groupby("run_id")["reward_non_neg1"].sum().rename("total_score")

    result = (
        pd.concat([total_rows, total_score], axis=1)
        .assign(success_rate=lambda d: d["total_score"] / d["total_rows"])
        .reset_index()
        .sort_values("run_id")
    )
    return result

df_subset = df[df["task_id"].isin(eval_task_ids)]
result_subset = agg_by_run_id(df).rename(columns={
    "metric": "metric_subset",
    "total_score": "total_score",
    "total_rows": "total_rows_subset",
})

result_subset

In [22]:
result_subset = agg_by_run_id(df_subset).rename(columns={
    "metric": "metric_subset",
    "total_score": "total_score",
    "total_rows": "total_rows_subset",
})
result_subset

NameError: name 'agg_by_run_id' is not defined